## Hugging Face Data setup for CLM

In [22]:
import sys
import datasets
import torch
from itertools import chain
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer
from transformers import default_data_collator

In [28]:
raw_datasets = load_dataset("tweet_eval", "stance_abortion")
tokenizer=AutoTokenizer.from_pretrained("bert-base-cased")

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/587 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/280 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/66 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /users2/avirinchipur/.cache/huggingface/datasets/tweet_eval/stance_abortion/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [29]:
print (raw_datasets['train'].column_names)
print (raw_datasets['train'].features)

['text', 'label']
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['none', 'against', 'favor'], id=None)}


In [30]:
raw_datasets['train'][0]

{'text': 'we remind ourselves that love means to be willing to give until it hurts - Mother Teresa',
 'label': 1}

In [31]:
text_column_name = "text"

def tokenize_function(examples):
    return tokenizer(examples[text_column_name])

tokenized_datasets = raw_datasets.map(
            tokenize_function,
            batched=True,
            num_proc=2,
            remove_columns=raw_datasets["train"].column_names,
            desc="Running tokenizer on dataset",
        )

Running tokenizer on dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

In [32]:
print (tokenized_datasets['train'].features)

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}


In [33]:
tokenized_datasets['train'][0]

{'input_ids': [101,
  1195,
  11484,
  9655,
  1115,
  1567,
  2086,
  1106,
  1129,
  4988,
  1106,
  1660,
  1235,
  1122,
  15483,
  118,
  4872,
  12575,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [51]:
examples

{'input_ids': [101,
  1195,
  11484,
  9655,
  1115,
  1567,
  2086,
  1106,
  1129,
  4988,
  1106,
  1660,
  1235,
  1122,
  15483,
  118,
  4872,
  12575,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [53]:
examples = tokenized_datasets['train'][0:5]
concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}

In [57]:
print (tokenizer.convert_ids_to_tokens(concatenated_examples['input_ids']))

['[CLS]', 'we', 'remind', 'ourselves', 'that', 'love', 'means', 'to', 'be', 'willing', 'to', 'give', 'until', 'it', 'hurts', '-', 'Mother', 'Teresa', '[SEP]', '[CLS]', '@', 'user', '@', 'user', 'and', 'most', 'Islanders', 'have', 'different', 'definitions', 'of', '"', 'time', '##ly', 'access', '"', '.', '#', 'irony', '#', 'p', '##ei', '##poli', '#', 'p', '##ei', '##vo', '##tes', '[SEP]', '[CLS]', 'Life', 'is', '#', 'precious', '&', 'so', 'are', 'babies', ',', 'mothers', ',', '&', 'fathers', '.', 'Please', 'support', 'the', 'sa', '##nc', '##ti', '##ty', 'of', 'Human', 'Life', '.', 'Think', '#', 'Se', '##m', '##ST', '[SEP]', '[CLS]', '@', 'user', 'too', 'many', 'people', 'are', 'taking', 'this', 'to', 'seriously', '#', 'Se', '##m', '##ST', '[SEP]', '[CLS]', 'Dude', 'i', 'won', 'a', '#', 'free', '##shirt', 'from', '@', 'user', '!', 'I', 'never', 'win', 'anything', 'lo', '##l', '#', 'A', '##bor', '##t', '##7', '##3', '#', 'Se', '##m', '##ST', '[SEP]']


In [44]:
block_size=128
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, and if the total_length < block_size  we exclude this batch and return an empty dict.
    # We could add padding if the model supported it instead of this drop, you can customize this part to your needs.
    total_length = max((total_length // block_size) * block_size, 1)
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [46]:
lm_datasets = tokenized_datasets.map(
            group_texts,
            batched=True,
            num_proc=2,
            desc=f"Grouping texts in chunks of {block_size}",
        )

Grouping texts in chunks of 128 #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Grouping texts in chunks of 128 #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Grouping texts in chunks of 128 #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Grouping texts in chunks of 128 #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Grouping texts in chunks of 128 #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Grouping texts in chunks of 128 #0:   0%|          | 0/1 [00:00<?, ?ba/s]

In [47]:
lm_datasets['train'][0].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [48]:
len(lm_datasets['train'][0]['input_ids']), len(tokenized_datasets['train'][0]['input_ids'])

(128, 19)

In [49]:
len(lm_datasets['train']), len(tokenized_datasets['train'])

(147, 587)

In [39]:
train_dataloader = DataLoader(
        lm_datasets['train'], shuffle=False, collate_fn=default_data_collator, batch_size=16
    )

In [40]:
# Get the first batch from the dataloader 
first_batch = next(iter(train_dataloader))

In [41]:
first_batch

{'input_ids': tensor([[  101,  1195, 11484,  ...,   102,   101,  2409],
         [11078,   112,  1325,  ...,  1135,   112,   188],
         [ 1136,  1139,  3026,  ...,   101,   160,  3048],
         ...,
         [  136,  3298,  4915,  ..., 16397,  1221,  1115],
         [ 1297,  3471,  1120,  ...,   119,  1195,   112],
         [ 1231,  1280,  1111,  ..., 10373,  2066,  1176]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]]),
 'labels': tensor([[  101,  1195, 11484,  ...,   102,   101,  2409],
         [11078,   112,  1325,  ...,  1135,   112,   188],
 

## Hugging Face Datasets

In [2]:
from datasets import Dataset

In [13]:
def gen():
    yield {"text": "Good", "answer":{"label": 0}}
    yield {"text": "Bad", "answer": {"label": 1}}
    yield {"text": "Ugly", "answer": {"label": 2}}
    yield {"text": "Marvelous", "answer":{"label": 3}}
    
ds = Dataset.from_generator(gen)

Using custom data configuration default-fd50a1c681ff73ef


Generating train split: 0 examples [00:00, ? examples/s]

Dataset generator downloaded and prepared to /users2/avirinchipur/.cache/huggingface/datasets/generator/default-fd50a1c681ff73ef/0.0.0. Subsequent calls will reuse this data.


In [14]:
ds

Dataset({
    features: ['text', 'answer'],
    num_rows: 4
})

In [17]:
from torch.utils.data import DataLoader
dl = DataLoader(ds, batch_size=2, shuffle=True)

In [18]:
f = next(iter(dl))

In [19]:
f

{'text': ['Good', 'Ugly'], 'answer': {'label': tensor([0, 2])}}

## Loading dict dataset into Huggingface Datasets

In [2]:
import torch

In [3]:
NUM_EXAMPLES = 4
BATCH_SEQ_LEN = 10
FEAT_DIM = 8
SEQ_LENS = torch.tensor([6, 3, 5, 10], dtype=torch.long)
torch.manual_seed(42)

embeddings = torch.randn(NUM_EXAMPLES, BATCH_SEQ_LEN, FEAT_DIM)
mask = torch.nn.utils.rnn.pad_sequence([torch.ones(seq_len) for seq_len in SEQ_LENS], batch_first=True)
print (embeddings.shape, mask.shape)
labels_1 = torch.randint(0, 2, (NUM_EXAMPLES, ), dtype=torch.long)
labels_2 = torch.randint(0, 2, (NUM_EXAMPLES, BATCH_SEQ_LEN), dtype=torch.long)

torch.Size([4, 10, 8]) torch.Size([4, 10])


In [31]:
dataset_dict = {
                'embedding':[embeddings[i].unsqueeze(0) for i in range(len(embeddings))], 
                'label':[labels_1[i] for i in range(len(labels_1))], 
                'id':[i for i in range(len(embeddings))]
                }

In [5]:
from datasets import Dataset, DatasetDict

In [6]:
ds = Dataset.from_dict(dataset_dict)

In [7]:
ds

Dataset({
    features: ['embedding', 'label', 'id'],
    num_rows: 4
})

In [8]:
ds.features

{'embedding': Sequence(feature=Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None), length=-1, id=None),
 'label': Value(dtype='int64', id=None),
 'id': Value(dtype='int64', id=None)}

In [9]:
# Load train, val and test ses into Dataset_dict object with embeddings, cls and label features
# Write map functions to be able to set the data up for different kinds of training strategies

In [10]:
ds_new = ds.train_test_split(test_size=0.2, shuffle=True, seed=42)

In [11]:
ds_new

DatasetDict({
    train: Dataset({
        features: ['embedding', 'label', 'id'],
        num_rows: 3
    })
    test: Dataset({
        features: ['embedding', 'label', 'id'],
        num_rows: 1
    })
})

In [18]:
from datasets import Sequence, ClassLabel, Features
from torch.utils.data import DataLoader
from torch import FloatTensor, IntTensor

In [91]:
def get_dataset(train_data:dict, val_data:dict, test_data:dict):
    """
        Returns the Huggingface datasets.dataset .
    """
    # TODO: Add support to infill default vector/tokens for missing timesteps
    dataset_obj = DatasetDict()
    dataset_obj['train'] = Dataset.from_dict(train_data)
    dataset_obj['dev'] = Dataset.from_dict(val_data)
    dataset_obj['test'] = Dataset.from_dict(test_data)
    return dataset_obj


def create_mask(examples):
    """
        Function that goes into DatasetDict.map() to create a mask pattern for the sequence
        This function is for applying MIL for a single label representing the sequence
    """
    def infill_missing_vector(instance):
        """
            Infills missing vector with the previous vector
            TODO: Other choices include default vector, neighbour average, learnable embedding vector
        """
        sorted_seq_nums = sorted(instance['seq_num'])
        missing_seq_nums = set(range(sorted_seq_nums[0], sorted_seq_nums[-1]+1)) - set(sorted_seq_nums)
        for seq_num in missing_seq_nums:
            instance['embeddings'][0].insert(seq_num, instance['embeddings'][0][seq_num-1])
            instance['seq_num'].insert(seq_num, seq_num)
        instance['labels'] = torch.tensor(instance['labels']).expand(1, len(instance['seq_num'])).tolist()
            # if 'cls' in instance:
                # instance['cls'][0].insert(seq_num, instance['cls'][0][seq_num-1])
        return instance

    def create_mask_pattern(instance):
        """
            Creates a mask pattern for the sequence
        """
        instance['mask'] = [[1]*len(instance['seq_num'])]# [1 if seq_num in instance['seq_num'] else 0 for seq_num in range(max(instance['seq_num'])+1)]
        return instance
    
    examples = infill_missing_vector(examples)
    examples = create_mask_pattern(examples)
    
    return examples  

In [92]:
NUM_EXAMPLES = 4
BATCH_SEQ_LEN = 10
FEAT_DIM = 8
SEQ_LENS = torch.tensor([6, 3, 5, 10], dtype=torch.long)
torch.manual_seed(420)

embeddings_train = [torch.randn(1, SEQ_LENS[i], FEAT_DIM) for i in range(NUM_EXAMPLES)]
labels_train = [torch.randint(0, 2, (1, 1), dtype=torch.long) for i in range(NUM_EXAMPLES)]
seq_nums_train = [torch.randperm(BATCH_SEQ_LEN)[:SEQ_LENS[i]].sort()[0] for i in range(NUM_EXAMPLES)]
seq_nums_train = [seq - seq[0] for seq in seq_nums_train]
# seq_nums_train = [torch.arange(SEQ_LENS[i], dtype=torch.long) for i in range(NUM_EXAMPLES)]

data_dict = dict(embeddings=embeddings_train, labels=labels_train, seq_num=seq_nums_train)
dataset = get_dataset(data_dict, data_dict, data_dict)

In [93]:
dataset_masked = dataset.map(create_mask)

  0%|          | 0/4 [00:00<?, ?ex/s]

  0%|          | 0/4 [00:00<?, ?ex/s]

  0%|          | 0/4 [00:00<?, ?ex/s]

In [94]:
i = 0
print ("Len of {}-th sequnce: {}".format(i, dataset_masked['train'][i]['seq_num'].__len__()))
print ("Shape of its embeddings object: {}".format(torch.tensor(dataset_masked['train'][0]['embeddings']).shape))
print ("Shape its mask object: {}".format(torch.tensor(dataset_masked['train'][0]['mask']).shape))
print ("Shape of its labels object: {}".format(torch.tensor(dataset_masked['train'][0]['labels']).shape))

Len of 0-th sequnce: 9
Shape of its embeddings object: torch.Size([1, 9, 8])
Shape its mask object: torch.Size([1, 9])
Shape of its labels object: torch.Size([1, 9])


In [95]:
dataset_masked['train'].features

{'embeddings': Sequence(feature=Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None), length=-1, id=None),
 'labels': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'seq_num': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'mask': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None)}

In [96]:
torch.tensor(dataset_masked['train']['embeddings'][0]).shape, torch.tensor(dataset_masked['train']['labels'][0]).shape, torch.tensor(dataset_masked['train']['seq_num'][0]).shape

(torch.Size([1, 9, 8]), torch.Size([1, 9]), torch.Size([9]))

In [99]:
for feat in dataset_masked['train']:
    print (feat['seq_num'])

[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7]
[0, 1, 2, 3, 4, 5, 6, 7]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [114]:
def collate_fn(features):
    # Features dict have embeddings, label, seq_num of single sequence
    # Embeddings shape: (1, seq_len, hidden_dim)
    # Labels shape: (1, seq_len)
    # seq_num shape: (seq_len, )
    first = features[0]
    batch = {}
    max_seq_len = max([feat['seq_num'][-1] for feat in features])+1
    for k, _ in first.items():
        if k == "embeddings":
            for feat in features:
                embeddings = torch.tensor(feat['embeddings']).clone()
                if embeddings.shape[1] < max_seq_len:
                    zeros = torch.zeros((1, max_seq_len - embeddings.shape[1], embeddings.shape[2]))
                    feat['embeddings'] = torch.cat((embeddings, zeros), dim=1)
            batch[k] = torch.cat([torch.tensor(f[k]) for f in features], dim=0)
        elif k == "seq_num":
            batch[k] = torch.nn.utils.rnn.pad_sequence([torch.tensor(f[k]) for f in features], padding_value=-1, batch_first=True)
        elif k == "labels":
            batch[k] = torch.nn.utils.rnn.pad_sequence([torch.tensor(f[k]).squeeze(0) for f in features], padding_value=-1, batch_first=True)
    return batch

dataloader = DataLoader(dataset_masked['train'], batch_size=2, shuffle=True, num_workers=0, collate_fn=collate_fn)

In [115]:
nxt = next(iter(dataloader))

9


/tmp/ipykernel_438/362737656.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch[k] = torch.cat([torch.tensor(f[k]) for f in features], dim=0)


In [116]:
nxt.keys()

dict_keys(['embeddings', 'labels', 'seq_num'])

In [117]:
nxt['embeddings'].shape

torch.Size([2, 9, 8])

In [118]:
nxt['seq_num']

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7, -1],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8]])

In [120]:
nxt['labels']

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0, -1],
        [ 1,  1,  1,  1,  1,  1,  1,  1,  1]])

In [135]:
nxt['embeddings'][1]

tensor([[ 0.7281, -0.7106, -0.6021,  0.9604,  0.4048, -1.3543, -0.4976,  0.4747],
        [-0.1976,  1.2683,  1.2243,  0.0981,  1.7423, -1.3527,  0.2191,  0.5526],
        [-0.6788,  0.5743,  0.1877, -0.3576, -0.3165,  0.5886, -0.8905,  0.4098],
        [ 1.9312,  1.0119, -1.4364, -1.1299, -0.1360,  1.6354,  0.6547,  0.5760],
        [ 1.1415,  0.0186, -1.8058,  0.9254, -0.3753,  1.0331, -0.6867,  0.6368],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

In [141]:
nxt['mask']

tensor([[1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 0]])

In [ ]:
#TODO: Fix the dimensions of the mask, seq_num and labels

In [113]:
torch.cat([embeddings[:1], torch.zeros(1, 2, 8)], axis=1).shape

torch.Size([1, 12, 8])